In [75]:
from urllib.request import urlopen
import pandas as pd
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import os
import plotly.express as px
from datetime import date
from datetime import timedelta

url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'
popUrl = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv'

confirmed_pd = pd.read_csv(url, index_col=False)
population_pd = pd.read_csv(popUrl, index_col=False)

sunday = date.today()
day = sunday.weekday()

while(day != 6):
    sunday = sunday - timedelta(days=1)
    day = sunday.weekday()

TE = sunday
TE = TE.strftime("%#m/%#d/%y")
TS = sunday - timedelta(days=7)
TS = TS.strftime("%#m/%#d/%y")
LE = sunday - timedelta(days=7)
LE = LE.strftime("%#m/%#d/%y")
LS = sunday - timedelta(days=14)
LS = LS.strftime("%#m/%#d/%y")

thisIR = []
lastIR = []
strFIPS = []
highIR = []
risk = []

population = population_pd['population']
lastStart = confirmed_pd[LS]
lastEnd = confirmed_pd[LE]
thisStart = confirmed_pd[TS]
thisEnd = confirmed_pd[TE]
FIPS = confirmed_pd['countyFIPS']
county = confirmed_pd['County Name'] + ', ' + confirmed_pd['State']

combined_pd = pd.DataFrame(list(zip(FIPS, county, population, lastStart, lastEnd, thisStart, thisEnd)), 
                           columns = ['FIPS', 'County', 'Population', 'Last Start', 'Last End', 'This Start','This End'])
combined_pd = combined_pd[combined_pd['Population'] != 0].reset_index(drop=True)

combined_pd['FIPS'] = combined_pd['FIPS'].astype('int64', copy=True)

for x in range(0,len(combined_pd)):
    thisIR.append((combined_pd['This End'][x] - combined_pd['This Start'][x])/combined_pd['Population'][x]*100000)
    lastIR.append((combined_pd['Last End'][x] - combined_pd['Last Start'][x])/combined_pd['Population'][x]*100000)
    if thisIR[x] > lastIR[x]:
        highIR.append(thisIR[x])
    else: 
        highIR.append(lastIR[x])
    strFIPS.append(str(combined_pd['FIPS'][x]).rjust(5,'0'))

combined_pd['FIPS'] = strFIPS
combined_pd['Last IR'] = lastIR
combined_pd['This IR'] = thisIR
combined_pd['Incidence Rate'] = highIR

mean = combined_pd['This IR'].mean()
std = combined_pd['This IR'].std()

for x in range(0,len(combined_pd)):
    if combined_pd['Incidence Rate'][x]<mean + std:
        risk.append('LOW')
    elif combined_pd['Incidence Rate'][x]>=mean + std and combined_pd['Incidence Rate'][x]<=mean+2*std:
        risk.append('MEDIUM')
    elif combined_pd['Incidence Rate'][x]>mean+2*std:
        risk.append('HIGH')
    else: 
        risk.append('ERROR')

combined_pd['Risk'] = risk
        
colorscale = ["#CCFFCC","#00FF00","#99FF00","#CCFF00","#FFFF00","#FFCC00","#FF6600","#FF0000"]

fig = px.choropleth(combined_pd, geojson=counties, locations='FIPS', color='Incidence Rate',
                    color_continuous_scale=colorscale,
                    range_color=(0, mean+std*2), 
                    scope="usa", title='Active Incidence Rate per County', hover_name = "County" , hover_data=['Risk', 'Incidence Rate'])

fig.update_traces(marker_line_width=.3, marker_opacity=.8,hovertemplate='<b>%{hovertext}</b><br>Risk: %{customdata[0]}<br><br>Incidence Rate: %{z}<extra></extra>')
fig.update_layout(annotations = [dict(text = 'Last Updated: ' + str((pd.datetime.today()).date()), x=.8, y=.91)],legend = dict(x=0.8),title_x = 0.4, font={"size":20, "color":"white"},geo=dict(bgcolor='#323130', lakecolor='#323130', subunitcolor='black'), plot_bgcolor='#111110', paper_bgcolor='#111110',margin={"r":0,"t":100,"l":0,"b":50})
fig.update_geos(showsubunits=True, subunitcolor='black')
print('working..')
os.chdir(r'C:\Users\borde\Documents\Incidence Map')
fig.write_html('index.html',auto_open=True)
print('done')

working..
done
